# CellOracle calculate pseudotime

calculate pseudotime using CellOracle's helper class

In [ ]:
import re
import logging as log
from pathlib import Path
import yaml

import numpy as np
import scipy as sp
import pandas as pd
import scanpy as sc
import celloracle as co
from celloracle.applications import Pseudotime_calculator

import matplotlib.pyplot as plt
from IPython.display import display, Markdown

In [ ]:
log.info(f"CellOracle version: {co.__version__}")

In [ ]:
plt.rcParams["figure.figsize"] = [5,5]
%config InlineBackend.figure_format = 'retina'
plt.rcParams["savefig.dpi"] = 300

%matplotlib inline

## Params

In [ ]:
## input
metacell_rna_h5ad = "/path/to/rna_metacells.h5ad"

## params
oracle_obj_path = "/path/to/celloracle.oracle"
embedding_key = "X_draw_graph_fa"
cluster_col = "cell_type_obs_column"
lineage_dictionary =  {"lineage": []}

## 1) Load

In [ ]:
log.info("load RNA anndata file")

In [ ]:
adata = sc.read_h5ad(metacell_rna_h5ad)

In [ ]:
lineage_dictionary = {
    k: v if v else adata.obs[cluster_col].unique().tolist()
    for k, v in lineage_dictionary.items()
}

In [ ]:
log.info("load celloracle object")

In [ ]:
oracle = co.load_hdf5(oracle_obj_path)

In [ ]:
pt = Pseudotime_calculator(oracle_object=oracle)

## 2) Add lineage information to pseudotime object

In [ ]:
log.info("add lineage information to pseudotime object")

In [ ]:
# Input lineage information into pseudotime object
pt.set_lineage(lineage_dictionary = lineage_dictionary)

# Visualize lineage information
pt.plot_lineages()

## 3) Select root cells of each lineage interactively

In [ ]:
try:
    import plotly.express as px
    def plot(adata, embedding_key, cluster_column_name):
        embedding = adata.obsm[embedding_key]
        df = pd.DataFrame(embedding, columns=["x", "y"])
        df["cluster"] = adata.obs[cluster_column_name].values
        df["label"] = adata.obs.index.values
        fig = px.scatter(df, x="x", y="y", hover_name=df["label"], color="cluster")
        fig.update_layout(
            autosize=False,
            width=1000,
            height=1000,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            paper_bgcolor="LightSteelBlue",
        )
        fig.show()
        
    pt.adata.obs["barcode"] = pt.adata.obs_names
    pt.adata.obs_names = range(len(pt.adata.obs_names))

    plot(adata = pt.adata,
         embedding_key = pt.obsm_key,
         cluster_column_name = pt.cluster_column_name)
except BaseException:
    log.exception("Plotly not found in your environment. Did you install plotly? Please read the instruction above.")

In [ ]:
pt.adata.obs_names = pt.adata.obs["barcode"]


### enter the number of selected cell here
pt.adata.obs["barcode"][1234]
##########################################

In [ ]:
# Estimated root cell name for each lineage
root_cells = {"lineage": "root_cell_barcode"}
pt.set_root_cells(root_cells=root_cells)

In [ ]:
log.info("plot root cells")
pt.plot_root_cells()

## 4) Calculate pseudotime

In [ ]:
if not "X_diffmap" in pt.adata.obsm:
    log.warning("no X_diffmap found in anndata, calculating diffusion map")
    sc.pp.neighbors(pt.adata, n_neighbors=30)
    sc.tl.diffmap(pt.adata)

In [ ]:
log.info("calculate pseudotime")

In [ ]:
# Calculate pseudotime
pt.get_pseudotime_per_each_lineage()

In [ ]:
# Check result
pt.adata.obs[["Pseudotime"]].head()

plot

In [ ]:
# Check results
pt.plot_pseudotime(cmap="rainbow")

## 5) Save

Add calculated pseudotime data to the oracle object

In [ ]:
oracle.adata.obs["Pseudotime"] = oracle.adata.obs.index.map(pt.adata.obs["Pseudotime"])

Save updated anndata object

In [ ]:
oracle.to_hdf5(oracle_obj_path)